In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier

# from xgboost import XGBClassifier
import pickle

import warnings
warnings.filterwarnings("ignore")

In [4]:
import random
random.seed(42)

In [5]:
df = pd.read_csv('./Data/url_updated.csv')

In [6]:
df['label'].value_counts(normalize=True)

0    0.5
1    0.5
Name: label, dtype: float64

In [7]:
x = df[['hostname_length','url_length',
       'path_length', 'fd_length', 'spcl_char', 'count-http','count-https', 'count-www', 'count-digits',
       'count-letters', 'count_dir','path_entropy']]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(x,y,random_state = 42, stratify = y )

In [8]:
ss = StandardScaler()

X_train_sc = ss.fit_transform(X_train)

X_test_sc = ss.transform(X_test)

In [9]:
def get_scores_sc(model):
    
    # instantiate model
    model = model()
    
    # fit to scaled data
    model.fit(X_train_sc, y_train)
    
    # make predictions
    preds = model.predict(X_test_sc)

    # print results
    print(f'{model} Training Score: {model.score(X_train_sc, y_train)}')
    print(f'{model} Testing Score: {model.score(X_test_sc, y_test)}')
    print()
    print(classification_report(y_test, preds))
    print(confusion_matrix(y_test, preds))

    return

In [10]:
def get_scores(model):
    
    # instantiate model
    model = model()
    
    # fit to data
    model.fit(X_train, y_train)
    
    # make predictions
    preds = model.predict(X_test)

    # print results
    print(f'{model} Training Score: {model.score(X_train, y_train)}')
    print(f'{model} Testing Score: {model.score(X_test, y_test)}')
    print()
    print(classification_report(y_test, preds))
    print(confusion_matrix(y_test, preds))

    return

In [11]:
get_scores(DecisionTreeClassifier)

DecisionTreeClassifier() Training Score: 0.999420297187282
DecisionTreeClassifier() Testing Score: 0.9945803057036433

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     79064
           1       0.99      0.99      0.99     79063

    accuracy                           0.99    158127
   macro avg       0.99      0.99      0.99    158127
weighted avg       0.99      0.99      0.99    158127

[[78615   449]
 [  408 78655]]


In [12]:
get_scores(BaggingClassifier)

KeyboardInterrupt: 

In [13]:
get_scores(RandomForestClassifier)

RandomForestClassifier() Training Score: 0.9994160811668258
RandomForestClassifier() Testing Score: 0.9967051800135335

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     79064
           1       1.00      1.00      1.00     79063

    accuracy                           1.00    158127
   macro avg       1.00      1.00      1.00    158127
weighted avg       1.00      1.00      1.00    158127

[[78906   158]
 [  363 78700]]


### Model Optimization

In [ ]:
# rfc = RandomForestClassifier()
# rfc_params = {'bootstrap': [True, False],
#                'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#                'max_features': ['auto', 'sqrt'],
#                'min_samples_leaf': [1, 2, 4],
#                'min_samples_split': [2, 5, 10],
#                'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600]}

# rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = rfc_params, cv = 3, verbose=2, random_state=42, 
#                                 n_jobs = -1)
# rfc_random.fit(X_train, y_train)
# print(f'Best Parameters: {rfc_random.best_params_}')

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [14]:
rfc = RandomForestClassifier(n_estimators = 1400, min_samples_split= 5, min_samples_leaf= 1, max_features= 'sqrt', max_depth= 90, 
                             bootstrap= True, n_jobs= -1, random_state=42)
rfc.fit(X_train, y_train)

preds = rfc.predict(X_test)
print(f'Training Score: {rfc.score(X_train, y_train)}')
print(f'Testing Score: {rfc.score(X_test, y_test)}')
print()
print(classification_report(y_test,preds))
print(confusion_matrix(y_test, preds))

cm = confusion_matrix(y_test, preds)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = rfc.classes_)
disp.plot(cmap = 'Reds', values_format='')

KeyboardInterrupt: 

In [15]:
# Create feature list
feature_list = list(x.columns)

# Get numerical feature importances
importances = list(rfc.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 3)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: hostname_length      Importance: 0.0
Variable: url_length           Importance: 0.0
Variable: path_length          Importance: 0.0
Variable: fd_length            Importance: 0.0
Variable: spcl_char            Importance: 0.0
Variable: count-http           Importance: 0.0
Variable: count-https          Importance: 0.0
Variable: count-www            Importance: 0.0
Variable: count-digits         Importance: 0.0
Variable: count-letters        Importance: 0.0
Variable: count_dir            Importance: 0.0
Variable: path_entropy         Importance: 0.0


[None, None, None, None, None, None, None, None, None, None, None, None]